In [1]:
import os, pickle
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ['openai_key']

In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [3]:
# Read in test data
with open('../data/sample_messages.pkl', 'rb') as file:
    messages = pickle.load(file)

# Filter for empty messages
messages = filter(lambda x: x['message'], messages)
# format messages like "<sender>: <text>"
formatted_messages = list(map(lambda x: f"<U>{x['sender']}</U>: {x['message']}", messages))
# Rearrange in correct order
formatted_messages = formatted_messages[::-1]

In [4]:
system_message = """
You will be given excerpts of messages from a group chat. 

The name of the user who sent a message is between the <U> </U> tags. 

Your task is to summarize the various conversations that took place in the group chat. For each conversation write a short summary of the points raised. 
"""

user_example = """
<U>David</U>: Wow the weather is really warm today
<U>David</U>: What are ya'll doing today pals. I'm going to cycling hohoho.
<U>James</U>: Ya sia today damn hot. Sweating like a dog bruh
<U>James</U>: Going out with my parents for lunch then watching a movie with YZ
<U>James</U>: WBU
<U>Gordon</U>: Ya it's too hot already. I need the aircon. 
<U>Gordon</U>: Man is going to study today. I got my final exam next week
<U>Gordon</U>: Can't wait to go out hahaha
"""

ai_example = "Everyone agrees the weather is extremely warm.\n\nDavid is going to spend the day cycling, James is lunching with his parents then watching a movie with YZ, and Gordon is studying for his final exams next week."

In [25]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", user_example),
        ("ai", ai_example),
        ("human", "{user_input}"),
    ]
)

In [26]:
prompt = "\n".join(formatted_messages)

In [27]:
prompt_input = chat_template.format_messages(user_input=prompt)

In [28]:
llm = ChatOpenAI(
    model_name = 'gpt-4'
    , openai_api_key = api_key
)

In [29]:
response = llm(prompt_input)